In [5]:
import damask
import numpy as np
import matplotlib.pyplot as plt

### Figure 2.1 Exemplary unitcells of bulk and film polycrystals.

In [3]:
N = 400 #total grain count in a unitcell
g = 288 #grid density g*g*g
air = 20 #thickness of virtual air

#generate polycrystal structure using Voronoi tessellation
geom = damask.Grid.from_Voronoi_tessellation(np.ones(3,dtype=int)*g,
                                            np.ones(3),
                                            np.random.random((N,3)))

#add a layer of virtual air to create a gap in the otherwise continuous metal bulk (surface normal parallel to y)
geom_air = geom.canvas(cells=geom.cells+np.array([0,air,0]),
                       offset=np.array([0,-air/2,0]))

#save
geom.save(f'{N}_{g}_bulk')
geom_air.save(f'{N}_{g}_film')

In [ ]:
#IPF colored grains
v = damask.VTK.load(f'{N}_{g}_bulk.vti')
material_ID = v.get('material').flatten()
IPF = np.ones((len(material_ID),3),np.uint8)

for i in range(N): #randomly assign crystallographic orientation to 400 grains
    IPF[np.where(material_ID==i)] = \
    np.uint8(damask.Orientation(damask.Rotation.from_random(), lattice='hP').IPF_color(np.array([0,0,1]))*255)

v.set('IPF_001', IPF).save(f'{N}_{g}_bulk_IPF')

In [7]:
#generate IPF color map
xp = []
yp = []
colorp = []
for i in range(60000): #points filling in the map
    o = damask.Orientation(damask.Rotation.from_random(), lattice='hP') #random orientation
    IPF = o.IPF_color(np.array([1,0,0])) #the IPF color of lab-x in this orientation
    o_SST = o.to_SST(np.array([1,0,0])) #transform to SST
    if o_SST[2] > 0:
        xp.append(o_SST[0]/(o_SST[2]+1))
        yp.append(o_SST[1]/(o_SST[2]+1))
        colorp.append(IPF)
plt.figure(figsize=(8, 4), dpi=80)
plt.scatter(xp, yp, c=colorp, marker='h')
plt.xlim(0, 1)
plt.ylim(0, 0.5)
plt.savefig('IPF_100.png')